#  LangChain의 RAG 콤포넌트 
- 벡터 저장소(Vector Store) 

### **학습 목표:**  임베딩 모델과 벡터 데이터베이스를 효과적으로 연동할 수 있다

### **실습 자료**: 해당 없음

---

# 환경 설정 및 준비

`(1) Env 환경변수`

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

`(2) 기본 라이브러리`

In [2]:
import os
from glob import glob

from pprint import pprint
import json

# 벡터 저장소 (Vector Store)

- 개념:
    - 벡터화된 데이터를 효율적으로 저장하고 검색하기 위한 특수 데이터베이스 시스템
    - 텍스트나 이미지 등의 비정형 데이터를 고차원 벡터 공간에 매핑하여 저장
    - 유사도 기반 검색을 통해 의미적으로 가까운 데이터를 빠르게 검색 가능 

- LangChain의 벡터 저장소 종류:
    - **Chroma**: 경량화된 임베딩 데이터베이스로 로컬 개발에 적합
    - **FAISS**: Facebook AI가 개발한 고성능 유사도 검색 라이브러리
    - **Pinecone**: 완전 관리형 벡터 데이터베이스 서비스
    - Milvus: 분산 벡터 데이터베이스로 대규모 데이터 처리에 적합
    - PostgreSQL: pgvector 확장을 통해 벡터 저장 및 검색 기능을 제공

- 주요 기능:
    - 벡터 색인화: 효율적인 검색을 위한 데이터 구조화를 수행
    - 근접 이웃 검색: 주어진 쿼리와 가장 유사한 벡터들을 검색 
    - 메타데이터 관리: 벡터와 관련된 부가 정보를 함께 저장하고 검색

- 사용 사례:
    - 시맨틱 문서 검색: 문서의 의미를 이해하여 검색
    - 추천 시스템: 유사한 아이템을 추천
    - 중복 데이터 감지: 유사한 콘텐츠를 검색 
    - 질의응답 시스템: 관련 문서에서 답변을 생성하는데 필요한 근거를 검색 

### 1. **Chroma**

- 사용자 편의성이 우수한 오픈소스 벡터 저장소
- `langchain-chroma` 패키지 설치

`(1) 벡터 저장소 초기화`

In [3]:
# 벡터 저장소에 문서를 저장할 때 적용할 임베딩 모델
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
embeddings_model = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")

In [4]:
# 벡터 저장소 생성
from langchain_chroma import Chroma

chroma_db = Chroma(
    collection_name="ai_smaple_collection",
    embedding_function=embeddings_model,
    persist_directory="./chroma_db",
)

In [5]:
# 현재 저장된 컬렉션 데이터 확인
chroma_db.get()

{'ids': [],
 'embeddings': None,
 'documents': [],
 'uris': None,
 'included': ['metadatas', 'documents'],
 'data': None,
 'metadatas': []}

`(2) 벡터 저장소 관리`  

- 문서 추가: `vector_store.add_documents(documents, ids)`

In [6]:
from langchain_core.documents import Document

# 문서 데이터 - (텍스트, 소스)
documents = [
    ("인공지능은 컴퓨터 과학의 한 분야입니다.", "AI 개론"),
    ("머신러닝은 인공지능의 하위 분야입니다.", "AI 개론"),
    ("딥러닝은 머신러닝의 한 종류입니다.", "딥러닝 입문"),
    ("자연어 처리는 컴퓨터가 인간의 언어를 이해하고 생성하는 기술입니다.", "AI 개론"),
    ("컴퓨터 비전은 컴퓨터가 디지털 이미지나 비디오를 이해하는 방법을 연구합니다.", "딥러닝 입문")
]

# Document 객체 생성
doc_objects = []
for content, source in documents:
    doc = Document(
        page_content=content,
        metadata={"source": source},
    )
    doc_objects.append(doc)


# 순차적 ID 리스트 생성
doc_ids = [f"DOC_{i}" for i in range(1, len(doc_objects) + 1)]

# 문서를 벡터 저장소에 저장
added_doc_ids = chroma_db.add_documents(documents=doc_objects, ids=doc_ids)

# 벡터 저장소에 저장된 문서를 확인
print(f"{len(added_doc_ids)}개의 문서가 성공적으로 벡터 저장소에 추가되었습니다.")
print(added_doc_ids)

5개의 문서가 성공적으로 벡터 저장소에 추가되었습니다.
['DOC_1', 'DOC_2', 'DOC_3', 'DOC_4', 'DOC_5']


In [7]:
# 저장된 문서 검색
query = "인공지능과 머신러닝의 관계는?"
results = chroma_db.similarity_search(query, k=2)

print(f"\n쿼리: {query}")
print("가장 유사한 문서:")
for doc in results:
    print(f"- {doc.page_content} [출처: {doc.metadata['source']}]")


쿼리: 인공지능과 머신러닝의 관계는?
가장 유사한 문서:
- 머신러닝은 인공지능의 하위 분야입니다. [출처: AI 개론]
- 딥러닝은 머신러닝의 한 종류입니다. [출처: 딥러닝 입문]


In [8]:
# 현재 저장된 컬렉션 데이터 확인
chroma_db.get()

{'ids': ['DOC_1', 'DOC_2', 'DOC_3', 'DOC_4', 'DOC_5'],
 'embeddings': None,
 'documents': ['인공지능은 컴퓨터 과학의 한 분야입니다.',
  '머신러닝은 인공지능의 하위 분야입니다.',
  '딥러닝은 머신러닝의 한 종류입니다.',
  '자연어 처리는 컴퓨터가 인간의 언어를 이해하고 생성하는 기술입니다.',
  '컴퓨터 비전은 컴퓨터가 디지털 이미지나 비디오를 이해하는 방법을 연구합니다.'],
 'uris': None,
 'included': ['metadatas', 'documents'],
 'data': None,
 'metadatas': [{'source': 'AI 개론'},
  {'source': 'AI 개론'},
  {'source': '딥러닝 입문'},
  {'source': 'AI 개론'},
  {'source': '딥러닝 입문'}]}

- 문서 수정: `vector_store.update_document(document_id, document)`

In [9]:
# 업데이트할 문서 생성
updated_document_1 = Document(
    page_content="인공지능은 컴퓨터 과학의 핵심 분야 중 하나로, 기계학습과 딥러닝을 포함합니다.",
    metadata={"source": "AI 개론"},
)

updated_document_2 = Document(
    page_content="머신러닝은 데이터로부터 학습하여 예측과 결정을 내리는 인공지능의 하위 분야입니다.",
    metadata={"source": "AI 개론"},
)

updated_document_3 = Document(
    page_content="딥러닝은 머신러닝의 한 종류로, 심층 신경망을 사용하여 학습합니다.",
    metadata={"source": "딥러닝 입문"},
)


# 단일 문서 업데이트
chroma_db.update_document(document_id="DOC_1", document=updated_document_1)

# 여러 문서 한 번에 업데이트
chroma_db.update_documents(
    ids=["DOC_2", "DOC_3"],
    documents=[updated_document_2, updated_document_3]
)

print("문서 업데이트 완료")

문서 업데이트 완료


In [10]:
# 저장된 문서 검색 예시
query = "인공지능과 머신러닝의 관계는?"
results = chroma_db.similarity_search(query, k=2)

print(f"\n쿼리: {query}")
print("가장 유사한 문서:")
for doc in results:
    print(f"- {doc.page_content} [출처: {doc.metadata['source']}]")


쿼리: 인공지능과 머신러닝의 관계는?
가장 유사한 문서:
- 머신러닝은 데이터로부터 학습하여 예측과 결정을 내리는 인공지능의 하위 분야입니다. [출처: AI 개론]
- 인공지능은 컴퓨터 과학의 핵심 분야 중 하나로, 기계학습과 딥러닝을 포함합니다. [출처: AI 개론]


- 문서 삭제: `vector_store.delete(ids)`

In [11]:
# 문서 id를 지정하여 삭제
chroma_db.delete(ids=["DOC_5"])

In [12]:
# 컬렉션 확인
chroma_db.get()

{'ids': ['DOC_1', 'DOC_2', 'DOC_3', 'DOC_4'],
 'embeddings': None,
 'documents': ['인공지능은 컴퓨터 과학의 핵심 분야 중 하나로, 기계학습과 딥러닝을 포함합니다.',
  '머신러닝은 데이터로부터 학습하여 예측과 결정을 내리는 인공지능의 하위 분야입니다.',
  '딥러닝은 머신러닝의 한 종류로, 심층 신경망을 사용하여 학습합니다.',
  '자연어 처리는 컴퓨터가 인간의 언어를 이해하고 생성하는 기술입니다.'],
 'uris': None,
 'included': ['metadatas', 'documents'],
 'data': None,
 'metadatas': [{'source': 'AI 개론'},
  {'source': 'AI 개론'},
  {'source': '딥러닝 입문'},
  {'source': 'AI 개론'}]}

`(3) 문서 검색`  

- 유사도 검색
    - 주어진 쿼리와 가장 유사한 문서를 반환
    -  k=2는 상위 2개의 결과를 반환하도록 지정
    - filter를 사용하여 특정 출처의 문서만 검색 가능

In [13]:
query = "인공지능과 머신러닝의 차이점은 무엇인가요?"
results = chroma_db.similarity_search(
    query,
    k=2,
    filter={"source": "AI 개론"}
)

print("유사도 검색 결과:")
for doc in results:
    print(f"- {doc.page_content} [출처: {doc.metadata['source']}]")

유사도 검색 결과:
- 머신러닝은 데이터로부터 학습하여 예측과 결정을 내리는 인공지능의 하위 분야입니다. [출처: AI 개론]
- 인공지능은 컴퓨터 과학의 핵심 분야 중 하나로, 기계학습과 딥러닝을 포함합니다. [출처: AI 개론]


- 유사도 점수가 포함된 검색
    - 유사도 점수를 함께 반환
    - 점수가 낮을수록 더 유사한 것을 의미 (거리 기준으로 점수가 산정되기 때문)

In [14]:
query = "딥러닝은 어떤 분야에서 사용되나요?"
results = chroma_db.similarity_search_with_score(
    query,
    k=2,
    filter={"source": "AI 개론"}
)

print("점수가 포함된 유사도 검색 결과:\n")
for doc, score in results:
    print(f"- 점수: {score:.4f}")
    print(f"  내용: {doc.page_content}")
    print(f"  [출처: {doc.metadata['source']}]")
    print()

점수가 포함된 유사도 검색 결과:

- 점수: 0.7292
  내용: 인공지능은 컴퓨터 과학의 핵심 분야 중 하나로, 기계학습과 딥러닝을 포함합니다.
  [출처: AI 개론]

- 점수: 0.8394
  내용: 머신러닝은 데이터로부터 학습하여 예측과 결정을 내리는 인공지능의 하위 분야입니다.
  [출처: AI 개론]



- 관련성 점수가 포함된 검색
    - 문서와 함께 0에서 1 사이의 관련성 점수를 반환
    - 0은 가장 관련성이 낮고, 1은 가장 관련성이 높음을 의미

In [15]:
query = "딥러닝은 어떤 분야에서 사용되나요?"
results = chroma_db.similarity_search_with_relevance_scores(
    query,
    k=2,
    filter={"source": "AI 개론"}
)

print(f"쿼리: {query}")
print("\n검색 결과 (관련성 점수 포함):")
for doc, score in results:
    print(f"- 관련성 점수: {score:.4f}")
    print(f"  내용: {doc.page_content}")
    print(f"  [출처: {doc.metadata['source']}]")
    print()

쿼리: 딥러닝은 어떤 분야에서 사용되나요?

검색 결과 (관련성 점수 포함):
- 관련성 점수: 0.4844
  내용: 인공지능은 컴퓨터 과학의 핵심 분야 중 하나로, 기계학습과 딥러닝을 포함합니다.
  [출처: AI 개론]

- 관련성 점수: 0.4065
  내용: 머신러닝은 데이터로부터 학습하여 예측과 결정을 내리는 인공지능의 하위 분야입니다.
  [출처: AI 개론]



`(4) 벡터 저장소 로드`  

In [16]:
chroma_db2 = Chroma(
    collection_name="ai_smaple_collection",
    embedding_function=embeddings_model,
    persist_directory="./chroma_db",
)

In [17]:
# 미리 임베딩된 쿼리 벡터를 사용하여 검색
query = "딥러닝은 어떤 분야에서 사용되나요?"
results = chroma_db2.similarity_search_with_relevance_scores(
    query,
    k=2,
    filter={"source": "딥러닝 입문"}
)

print(f"쿼리: {query}")
print("\n검색 결과 (관련성 점수 포함):")
for doc, score in results:
    print(f"- 관련성 점수: {score:.4f}")
    print(f"  내용: {doc.page_content}")
    print(f"  [출처: {doc.metadata['source']}]")
    print()

쿼리: 딥러닝은 어떤 분야에서 사용되나요?

검색 결과 (관련성 점수 포함):
- 관련성 점수: 0.5919
  내용: 딥러닝은 머신러닝의 한 종류로, 심층 신경망을 사용하여 학습합니다.
  [출처: 딥러닝 입문]



### **2 FAISS(Facebook AI Similarity Search)**

- 효율적인 벡터 유사도 검색 및 클러스터링을 위한 오픈소스 벡터 저장소 
- `faiss-cpu` 패키지 설치

`(1) 벡터 저장소 초기화`

In [18]:
# 벡터 저장소 생성
import faiss 
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS

# FAISS 인덱스 초기화 (유클리드 거리 사용)
faiss_index = faiss.IndexFlatL2(len(embeddings_model.embed_query("hello world")))
print("FAISS 인덱스 초기화 완료")

FAISS 인덱스 초기화 완료


In [ ]:
# FAISS 벡터 저장소의 벡터 차원 수 (임베딩 차원 수)
faiss_index.d

In [19]:
# FAISS 벡터 저장소 생성
faiss_db = FAISS(
    embedding_function=embeddings_model,
    index=faiss_index,           # 벡터 검색을 위한 데이터 구조를 정의
    docstore=InMemoryDocstore(), # 문서 저장소 객체를 지정 - 문서의 원본 내용과 메타데이터를 보관
    index_to_docstore_id={},     # 인덱스와 문서 간의 연결을 관리 (매핑 딕셔너리)
)
# 저장된 문서의 갯수 확인
faiss_db.index.ntotal

0

`(2) 벡터 저장소 관리`  

- 문서 추가: `vector_store.add_documents(documents, ids)`

In [20]:
from langchain_core.documents import Document

# 문서 데이터 - (텍스트, 소스)
documents = [
    ("인공지능은 컴퓨터 과학의 한 분야입니다.", "AI 개론"),
    ("머신러닝은 인공지능의 하위 분야입니다.", "AI 개론"),
    ("딥러닝은 머신러닝의 한 종류입니다.", "딥러닝 입문"),
    ("자연어 처리는 컴퓨터가 인간의 언어를 이해하고 생성하는 기술입니다.", "AI 개론"),
    ("컴퓨터 비전은 컴퓨터가 디지털 이미지나 비디오를 이해하는 방법을 연구합니다.", "딥러닝 입문")
]

# Document 객체 생성
doc_objects = []
for content, source in documents:
    doc = Document(
        page_content=content,
        metadata={"source": source},
    )
    doc_objects.append(doc)


# 순차적 ID 리스트 생성
doc_ids = [f"DOC_{i}" for i in range(1, len(doc_objects) + 1)]

# 문서를 벡터 저장소에 저장
added_doc_ids = faiss_db.add_documents(documents=doc_objects, ids=doc_ids)

# 벡터 저장소에 저장된 문서를 확인
print(f"{len(added_doc_ids)}개의 문서가 성공적으로 벡터 저장소에 추가되었습니다.")
print(added_doc_ids)

5개의 문서가 성공적으로 벡터 저장소에 추가되었습니다.
['DOC_1', 'DOC_2', 'DOC_3', 'DOC_4', 'DOC_5']


In [21]:
# 저장된 문서의 갯수 확인
faiss_db.index.ntotal

5

In [22]:
# 저장된 인덱스 확인
faiss_db.index_to_docstore_id

{0: 'DOC_1', 1: 'DOC_2', 2: 'DOC_3', 3: 'DOC_4', 4: 'DOC_5'}

In [23]:
# 저장된 문서 검색
faiss_db.docstore.search('DOC_1')

Document(id='DOC_1', metadata={'source': 'AI 개론'}, page_content='인공지능은 컴퓨터 과학의 한 분야입니다.')

- 문서 삭제: `vector_store.delete(ids)`

In [24]:
# 문서 id를 지정하여 삭제
faiss_db.delete(ids=["DOC_5"])

True

In [25]:
# 컬렉션 확인
faiss_db.index.ntotal

4

In [26]:
# 저장된 인덱스 확인
faiss_db.index_to_docstore_id

{0: 'DOC_1', 1: 'DOC_2', 2: 'DOC_3', 3: 'DOC_4'}

In [27]:
# 저장된 문서 객체를 확인
faiss_db.docstore._dict

{'DOC_1': Document(id='DOC_1', metadata={'source': 'AI 개론'}, page_content='인공지능은 컴퓨터 과학의 한 분야입니다.'),
 'DOC_2': Document(id='DOC_2', metadata={'source': 'AI 개론'}, page_content='머신러닝은 인공지능의 하위 분야입니다.'),
 'DOC_3': Document(id='DOC_3', metadata={'source': '딥러닝 입문'}, page_content='딥러닝은 머신러닝의 한 종류입니다.'),
 'DOC_4': Document(id='DOC_4', metadata={'source': 'AI 개론'}, page_content='자연어 처리는 컴퓨터가 인간의 언어를 이해하고 생성하는 기술입니다.')}

`(3) 문서 검색`  

- 유사도 검색
    - 주어진 쿼리와 가장 유사한 문서를 반환
    - k=2는 상위 2개의 결과를 반환하도록 지정
    - filter를 사용하여 특정 출처의 문서만 검색 가능

In [28]:
query = "인공지능과 머신러닝의 차이점은 무엇인가요?"
results = faiss_db.similarity_search(
    query,
    k=2,
    filter={"source": "AI 개론"}
)

print("유사도 검색 결과:")
for doc in results:
    print(f"- {doc.page_content} [출처: {doc.metadata['source']}]")

유사도 검색 결과:
- 머신러닝은 인공지능의 하위 분야입니다. [출처: AI 개론]
- 인공지능은 컴퓨터 과학의 한 분야입니다. [출처: AI 개론]


- 유사도 점수가 포함된 검색
    - 유사도 점수를 함께 반환
    - 점수가 낮을수록 더 유사한 것을 의미

In [29]:
query = "딥러닝은 어떤 분야에서 사용되나요?"
results = faiss_db.similarity_search_with_score(
    query,
    k=2,
    filter={"source": "AI 개론"}
)

print("점수가 포함된 유사도 검색 결과:\n")
for doc, score in results:
    print(f"- 점수: {score:.4f}")
    print(f"  내용: {doc.page_content}")
    print(f"  [출처: {doc.metadata['source']}]")
    print()

점수가 포함된 유사도 검색 결과:

- 점수: 0.8442
  내용: 머신러닝은 인공지능의 하위 분야입니다.
  [출처: AI 개론]

- 점수: 0.9845
  내용: 인공지능은 컴퓨터 과학의 한 분야입니다.
  [출처: AI 개론]



- 관련성 점수가 포함된 검색
    - 문서와 함께 0에서 1 사이의 관련성 점수를 반환
    - 0은 가장 관련성이 낮고, 1은 가장 관련성이 높음을 의미

In [30]:
query = "딥러닝은 어떤 분야에서 사용되나요?"
results = faiss_db.similarity_search_with_relevance_scores(
    query,
    k=2,
    filter={"source": "AI 개론"}
)

print(f"쿼리: {query}")
print("\n검색 결과 (관련성 점수 포함):")
for doc, score in results:
    print(f"- 관련성 점수: {score:.4f}")
    print(f"  내용: {doc.page_content}")
    print(f"  [출처: {doc.metadata['source']}]")
    print()

쿼리: 딥러닝은 어떤 분야에서 사용되나요?

검색 결과 (관련성 점수 포함):
- 관련성 점수: 0.4031
  내용: 머신러닝은 인공지능의 하위 분야입니다.
  [출처: AI 개론]

- 관련성 점수: 0.3038
  내용: 인공지능은 컴퓨터 과학의 한 분야입니다.
  [출처: AI 개론]



`(4) 로컬에 저장 및 로드`  

In [31]:
# 로컬에 저장
faiss_db.save_local("faiss_ai_smaple_index")

In [32]:
# 로컬에 저장된 FAISS 벡터 저장소 불러오기
faiss_db2 = FAISS.load_local(
    "faiss_ai_smaple_index", embeddings_model, allow_dangerous_deserialization=True
)

In [33]:
query = "딥러닝은 어떤 분야에서 사용되나요?"
results = faiss_db2.similarity_search_with_relevance_scores(
    query,
    k=2,
    filter={"source": "딥러닝 입문"}
)

print(f"쿼리: {query}")
print("\n검색 결과 (관련성 점수 포함):")
for doc, score in results:
    print(f"- 관련성 점수: {score:.4f}")
    print(f"  내용: {doc.page_content}")
    print(f"  [출처: {doc.metadata['source']}]")
    print()

쿼리: 딥러닝은 어떤 분야에서 사용되나요?

검색 결과 (관련성 점수 포함):
- 관련성 점수: 0.5392
  내용: 딥러닝은 머신러닝의 한 종류입니다.
  [출처: 딥러닝 입문]



### 3. **Pinecone**

- 상용 클라우드 기반의 벡터 데이터베이스 서비스 (SaaS)
- 실시간 고성능 벡터 검색 제공
- 회원가입 및 API 인증키 발급 (.env 파일에 `PINECONE_API_KEY` 등록)
- `langchain-pinecone pinecone-notebooks` 패키지 설치

`(1) 환경 설정`

In [42]:
# PINECONE_API_KEY 환경 변수 설정 후에 실행
from dotenv import load_dotenv
load_dotenv()

True

In [44]:
# 인증 설정
import os
from pinecone import Pinecone, ServerlessSpec 
pinecone_api_key = os.environ.get("PINECONE_API_KEY","pcsk_5gmQZZ_TqHS12RFTYs6T18GEzAVdoukwk58svhgWaaSJCrTqJzaumZtjP8Z6WWVk7bSots")
pc = Pinecone(api_key="pcsk_5gmQZZ_TqHS12RFTYs6T18GEzAVdoukwk58svhgWaaSJCrTqJzaumZtjP8Z6WWVk7bSots")

In [45]:
# 기존 인덱스 리스트 확인
existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]
print(f"기존 인덱스: {existing_indexes}")

기존 인덱스: []


`(2) 벡터 저장소 초기화`

In [46]:
import time 

# 인데스 이름 설정
index_name = "ai-smaple-index"

# 인덱스가 없으면 생성
if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=1024,
        metric="euclidean",  # 유사도 측정 방법 - euclidean, cosine, dotproduct
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)

# 인덱스 이름을 사용하여 인덱스 객체 생성
index = pc.Index(index_name)

In [47]:
# 인덱스 정보 확인
index_name = "ai-smaple-index"

index_info = pc.describe_index(index_name)
index_info

{
    "name": "ai-smaple-index",
    "metric": "euclidean",
    "host": "ai-smaple-index-mtxbeyj.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 1024,
    "deletion_protection": "disabled",
    "tags": null
}

In [48]:
from langchain_pinecone import PineconeVectorStore  

# PINECONE 벡터 저장소 생성
pinecone_db = PineconeVectorStore(index=index, embedding=embeddings_model)

# 벡터 저장소 객체 확인 
pinecone_db

h:\miniconda3\envs\modu\Lib\site-packages\langchain_pinecone\__init__.py:3: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from langchain_pinecone.vectorstores import Pinecone, PineconeVectorStore


In [49]:
# 저장된 문서의 갯수 확인
pinecone_db._index.describe_index_stats()

{'dimension': 1024,
 'index_fullness': 0.0,
 'metric': 'euclidean',
 'namespaces': {},
 'total_vector_count': 0,
 'vector_type': 'dense'}

`(2) 벡터 저장소 관리`  

- 문서 추가: `vector_store.add_documents(documents, ids)`

In [50]:
from langchain_core.documents import Document

# 문서 데이터 - (텍스트, 소스)
documents = [
    ("인공지능은 컴퓨터 과학의 한 분야입니다.", "AI 개론"),
    ("머신러닝은 인공지능의 하위 분야입니다.", "AI 개론"),
    ("딥러닝은 머신러닝의 한 종류입니다.", "딥러닝 입문"),
    ("자연어 처리는 컴퓨터가 인간의 언어를 이해하고 생성하는 기술입니다.", "AI 개론"),
    ("컴퓨터 비전은 컴퓨터가 디지털 이미지나 비디오를 이해하는 방법을 연구합니다.", "딥러닝 입문")
]

# Document 객체 생성
doc_objects = []
for content, source in documents:
    doc = Document(
        page_content=content,
        metadata={"source": source},
    )
    doc_objects.append(doc)


# 순차적 ID 리스트 생성
doc_ids = [f"DOC_{i}" for i in range(1, len(doc_objects) + 1)]

# 문서를 벡터 저장소에 저장
added_doc_ids = pinecone_db.add_documents(documents=doc_objects, ids=doc_ids)

# 벡터 저장소에 저장된 문서를 확인
print(f"{len(added_doc_ids)}개의 문서가 성공적으로 벡터 저장소에 추가되었습니다.")
print(added_doc_ids)

5개의 문서가 성공적으로 벡터 저장소에 추가되었습니다.
['DOC_1', 'DOC_2', 'DOC_3', 'DOC_4', 'DOC_5']


In [51]:
# 저장된 문서의 갯수 확인 - 동기화에 시간이 걸릴 수 있음
pinecone_db._index.describe_index_stats()

{'dimension': 1024,
 'index_fullness': 0.0,
 'metric': 'euclidean',
 'namespaces': {},
 'total_vector_count': 0,
 'vector_type': 'dense'}

In [52]:
# 저장된 문서 검색
query = "인공지능과 머신러닝의 관계는?"
results = pinecone_db.similarity_search(query, k=2)

print(f"\n쿼리: {query}")
print("가장 유사한 문서:")
for doc in results:
    print(f"- {doc.page_content} [출처: {doc.metadata['source']}]")


쿼리: 인공지능과 머신러닝의 관계는?
가장 유사한 문서:


- 문서 삭제: `vector_store.delete(ids)`

In [53]:
# 문서 id를 지정하여 삭제
pinecone_db.delete(ids=["DOC_5"])

In [54]:
# 컬렉션 확인
pinecone_db._index.describe_index_stats()

{'dimension': 1024,
 'index_fullness': 0.0,
 'metric': 'euclidean',
 'namespaces': {},
 'total_vector_count': 0,
 'vector_type': 'dense'}

`(3) 문서 검색`  

- 유사도 검색
    - 주어진 쿼리와 가장 유사한 문서를 반환
    -  k=2는 상위 2개의 결과를 반환하도록 지정
    - filter를 사용하여 특정 출처의 문서만 검색 가능

In [55]:
query = "인공지능과 머신러닝의 차이점은 무엇인가요?"
results = pinecone_db.similarity_search(
    query,
    k=2,
    filter={"source": "AI 개론"}
)

print("유사도 검색 결과:")
for doc in results:
    print(f"- {doc.page_content} [출처: {doc.metadata['source']}]")

유사도 검색 결과:


- 유사도 점수가 포함된 검색
    - 유사도 점수를 함께 반환
    - 점수가 낮을수록 더 유사한 것을 의미 (거리 기준으로 점수가 산정되기 때문)

In [56]:
query = "딥러닝은 어떤 분야에서 사용되나요?"
results = pinecone_db.similarity_search_with_score(
    query,
    k=2,
    filter={"source": "AI 개론"}
)

print("점수가 포함된 유사도 검색 결과:\n")
for doc, score in results:
    print(f"- 점수: {score:.4f}")
    print(f"  내용: {doc.page_content}")
    print(f"  [출처: {doc.metadata['source']}]")
    print()

점수가 포함된 유사도 검색 결과:



- 관련성 점수가 포함된 검색
    - 문서와 함께 0에서 1 사이의 관련성 점수를 반환
    - 0은 가장 관련성이 낮고, 1은 가장 관련성이 높음을 의미

In [57]:
query = "딥러닝은 어떤 분야에서 사용되나요?"
results = pinecone_db.similarity_search_with_relevance_scores(
    query,
    k=2,
    filter={"source": "AI 개론"}
)

print(f"쿼리: {query}")
print("\n검색 결과 (관련성 점수 포함):")
for doc, score in results:
    print(f"- 관련성 점수: {score:.4f}")
    print(f"  내용: {doc.page_content}")
    print(f"  [출처: {doc.metadata['source']}]")
    print()

쿼리: 딥러닝은 어떤 분야에서 사용되나요?

검색 결과 (관련성 점수 포함):


`(4) 벡터 저장소 로드`  

In [58]:
# 저장된 인덱스 확인해서 초기화 
index_name = "ai-smaple-index"
index = pc.Index(index_name)
pinecone_db2 = PineconeVectorStore(index=index, embedding=embeddings_model)

# 저장된 문서 정보를 확인
pinecone_db2._index.describe_index_stats()

{'dimension': 1024,
 'index_fullness': 0.0,
 'metric': 'euclidean',
 'namespaces': {},
 'total_vector_count': 0,
 'vector_type': 'dense'}

In [59]:
# 미리 임베딩된 쿼리 벡터를 사용하여 검색
query = "딥러닝은 어떤 분야에서 사용되나요?"
results = pinecone_db2.similarity_search_with_relevance_scores(
    query,
    k=2,
    filter={"source": "AI 개론"}
)

print(f"쿼리: {query}")
print("\n검색 결과 (관련성 점수 포함):")
for doc, score in results:
    print(f"- 관련성 점수: {score:.4f}")
    print(f"  내용: {doc.page_content}")
    print(f"  [출처: {doc.metadata['source']}]")
    print()

쿼리: 딥러닝은 어떤 분야에서 사용되나요?

검색 결과 (관련성 점수 포함):


# [실습 프로젝트]

1. 아래 샘플 문서들을 벡터 저장소에 저장하는 코드를 작성합니다. 
   - 적절한 벡터 저장소 선택 
   - 임베딩 모델 설정
   - 문서 구조 설계 (metadata 정의)

2. 벡터 저장소를 사용하여 다음 기능을 구현합니다. 
   - 새로운 문서 추가
   - 문서 삭제
   - 문서 검색: 유사도 점수 계산, 메타데이터 기반 필터링 등 

In [60]:
# 샘플 문서 데이터 
documents = [
    {"content": "인공지능 기술의 발전과 미래", "type": "article", "author": "김철수"},
    {"content": "데이터 분석 입문 가이드", "type": "tutorial", "author": "이영희"},
    {"content": "머신러닝 모델 성능 개선 방법", "type": "research", "author": "박지성"},
    {"content": "블록체인 기술과 금융 혁신", "type": "article", "author": "정민우"},
    {"content": "클라우드 컴퓨팅 아키텍처 설계", "type": "tutorial", "author": "강다은"},
    {"content": "사이버 보안 위협 대응 전략", "type": "research", "author": "홍길동"},
    {"content": "빅데이터 처리 시스템 구축 사례", "type": "article", "author": "송지원"},
    {"content": "웹 개발자를 위한 REST API 가이드", "type": "tutorial", "author": "임성준"},
    {"content": "자연어 처리 알고리즘 비교 연구", "type": "research", "author": "최유진"},
    {"content": "디지털 트랜스포메이션 성공 전략", "type": "article", "author": "백승호"},
    {"content": "파이썬으로 시작하는 데이터 시각화", "type": "tutorial", "author": "유미란"},
    {"content": "강화학습을 활용한 게임 AI 개발", "type": "research", "author": "조현우"},
    {"content": "5G 네트워크 기술 동향", "type": "article", "author": "윤서연"},
    {"content": "도커 컨테이너 실전 가이드", "type": "tutorial", "author": "장민석"},
    {"content": "추천 시스템 최적화 연구", "type": "research", "author": "신영수"},
    {"content": "스마트 시티 구현 기술", "type": "article", "author": "권태영"},
    {"content": "깃허브 활용 협업 가이드", "type": "tutorial", "author": "오지훈"},
    {"content": "컴퓨터 비전 응용 사례 연구", "type": "research", "author": "남궁민"},
    {"content": "양자 컴퓨팅의 현재와 미래", "type": "article", "author": "하은주"},
    {"content": "리액트 네이티브 앱 개발 입문", "type": "tutorial", "author": "문동현"},
    {"content": "음성인식 시스템 성능 평가", "type": "research", "author": "심준호"},
    {"content": "메타버스 플랫폼 개발 동향", "type": "article", "author": "류아린"},
    {"content": "NoSQL 데이터베이스 설계 패턴", "type": "tutorial", "author": "반승현"},
    {"content": "엣지 컴퓨팅 적용 사례 연구", "type": "research", "author": "주민정"},
    {"content": "디지털 헬스케어 기술 혁신", "type": "article", "author": "구본우"},
    {"content": "마이크로서비스 아키텍처 구현", "type": "tutorial", "author": "염지현"},
    {"content": "강화학습 기반 로봇 제어 연구", "type": "research", "author": "탁현우"},
    {"content": "친환경 IT 인프라 구축 방안", "type": "article", "author": "방승미"},
    {"content": "프론트엔드 성능 최적화 기법", "type": "tutorial", "author": "곽준영"},
    {"content": "시계열 데이터 예측 모델 연구", "type": "research", "author": "추민서"}
]

In [61]:
# Document 객체 생성
from langchain_core.documents import Document

doc_objects = []

for i, doc in enumerate(documents): 
    doc_obj = Document(
        page_content=doc["content"],
        metadata={"type": doc["type"], "author": doc["author"]},
    )
    doc_objects.append(doc_obj)

# uuid 생성
import uuid

doc_ids = [str(uuid.uuid4()) for _ in range(len(doc_objects))]

### 여기에 나머지 코드를 작성하세요 ###



In [63]:
# ...existing code...

# 벡터 저장소에 문서 저장 및 관리 기능 구현 (Chroma 사용)
try:
    chroma_db  # reuse if already defined in notebook
except NameError:
    from langchain_chroma import Chroma
    try:
        embeddings_model  # reuse if already defined
    except NameError:
        from langchain_huggingface.embeddings import HuggingFaceEmbeddings
        embeddings_model = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")
    chroma_db = Chroma(
        collection_name="project_documents_collection",
        embedding_function=embeddings_model,
        persist_directory="./chroma_db_project",
    )

# 문서 추가 (초기 일괄 추가)
added_ids = chroma_db.add_documents(documents=doc_objects, ids=doc_ids)
print(f"{len(added_ids)} documents added to Chroma (collection={chroma_db.collection_name})")

# 유틸리티 함수들: 추가, 삭제, 검색
from typing import List, Optional, Tuple
import uuid

def add_new_document(content: str, doc_type: str, author: str) -> str:
    """새 문서를 벡터 저장소에 추가하고 생성된 id 반환"""
    new_doc = Document(page_content=content, metadata={"type": doc_type, "author": author})
    new_id = str(uuid.uuid4())
    chroma_db.add_documents(documents=[new_doc], ids=[new_id])
    print(f"Added document id={new_id}")
    return new_id

def delete_documents_by_ids(ids: List[str]) -> None:
    """지정한 id 리스트를 벡터 저장소에서 삭제"""
    chroma_db.delete(ids=ids)
    print(f"Deleted ids: {ids}")

def search_documents(query: str, k: int = 3, filter: Optional[dict] = None, with_score: bool = False) -> List:
    """
    검색 함수.
    - with_score=False: Document 리스트 반환
    - with_score=True: List of (Document, score) 반환 (score: 거리 or 유사도 depending on backend)
    """
    if with_score:
        results = chroma_db.similarity_search_with_score(query, k=k, filter=filter)
    else:
        results = chroma_db.similarity_search(query, k=k, filter=filter)
    return results

# 사용 예시
# 1) 새로운 문서 추가
new_id = add_new_document("AI 기반 추천 시스템 설계 사례", "research", "테스트저자")

# 2) 검색 - 메타데이터 필터링 (type이 article인 문서 중에서)
query = "인공지능 기술"
results_filtered = search_documents(query, k=5, filter={"type": "article"}, with_score=False)
print(f"\nSearch results (filtered by type=article) for '{query}':")
for doc in results_filtered:
    print(f"- id: {getattr(doc, 'metadata', {}).get('id', 'N/A')}, content: {doc.page_content[:80]} , metadata: {doc.metadata}")

# 3) 검색 - 점수 포함 (상위 3개)
results_with_score = search_documents("추천 시스템", k=3, with_score=True)
print(f"\nSearch results with scores for '추천 시스템':")
for doc, score in results_with_score:
    print(f"- score: {score:.4f}, content: {doc.page_content[:80]}, metadata: {doc.metadata}")

# 4) 문서 삭제 (예시: 방금 추가한 문서 삭제)
delete_documents_by_ids([new_id])

# 영구 저장 (Chroma에 persist가 동작하는 경우)
try:
    chroma_db.persist()
    print("Chroma DB persisted to disk.")
except Exception:
    # 일부 langchain_chroma 버전은 persist()를 지원하지 않음
    pass

AttributeError: 'Chroma' object has no attribute 'collection_name'